# Análisis de datos utilizando PyRoot


###Cargo los paquetes nesesarias

In [1]:
#from IPython.core.interactiveshell import InteractiveShell
import ROOT
import os
import commands
#InteractiveShell.ast_node_interactivity = "all" # Para tener el modo interactivo completo

###Cargo las Librerías nesesarias

In [2]:
ROOT.gROOT.Reset()
#Esta es la ruta a la librería dinámica de Delphes (debe de ser compilada con la misma versión de root del notebook)
#Solo funciona con la versión de Delphes compilada con root 5 (y obiamente root 5)
ROOT.gSystem.AddDynamicPath("~/HEPTools/Delphes/delphes5/")
ROOT.gSystem.Load("libDelphes");

###Construcción de los TChain

In [3]:
#Lista de carpetas le analisis
Path1= "/scratch/cms/Datos"
Path2= "/Events"
Path3= "/tag_1_delphes_events.root"
ListOfFolders = ["/BackGround-DY_2j-Run0","/BackGround-tW-Run0","/BackGround-WW-Run0","/Signal_2j_mu-nu-Run0","/BackGround-mumuW"]


In [4]:
#Para guardar la info sobre los cortes
signal={};
for Folder in ListOfFolders:
    comand ="ls " + Path1 + Folder + Path2;
    tmp = commands.getstatusoutput( comand )
    
    #Parto la cadena de caracteres por '\n', y asigno las partes a un arreglo
    Runs = tmp[1].split("\n");
    
    #Creo el objeto TChain
    MainChain=ROOT.TChain("Delphes")
    
    #Lleno el objeto MainChain con las salidas de MadGraph-Pythia_Delphes
    for run in range(len(Runs)):
        imput = Path1 + Folder + Path2 + "/" + Runs[run] + "/tag_1_delphes_events.root"
        MainChain.Add(imput)
        
    #Creo el objeto ExRootTreeReader
    treeReader = ROOT.ExRootTreeReader(MainChain)
    numberOfEntries = treeReader.GetEntries()
    
    #Obtengo las ramas que voy a usar
    branchJet = treeReader.UseBranch("Jet")
    branchMuon = treeReader.UseBranch("Muon")
    branchMissingET = treeReader.UseBranch("MissingET")

    # Inicializo los contadores
    cut1=0; cut2=0; cut3=0; cut4=0; cut5=0;
    cuts={};
    
    # Loop sobre los eventos
    for entry in range(0, numberOfEntries):
        #Se carga la entrada espesífica
        treeReader.ReadEntry(entry)
        
        # Primer corte, al menos 2 jet
        if branchJet.GetEntries() > 1:
            cut1 = cut1 +1;
            bcount=0;
            for i in range(0, branchJet.GetEntries() ): #Esta parte es la que produce el error (TClass::TClass:0: RuntimeWarning: no dictionary for class CompBase is available)
                jet = branchJet.At(i)
                #Cuento cuantos jet hay identificados como b quarks
                if jet.BTag :
                    bcount = bcount + 1;
            
            # Segundo corte, al menos 2 jet b       
            if bcount > 1:
                cut2 = cut2 +1;
            
                # If event contains at least 2 muons
                if branchMuon.GetEntries() > 1:
                    cut3 = cut3 +1;
                    # Tomo los dos muones mas energéticos y comparo sus cargas eléctricas
                    muon1 = branchMuon.At(0)
                    muon2 = branchMuon.At(1)
                
                    #Corte 4: Que los muones mas energéticos tengan cargas diferentes
                    if muon1.Charge*muon2.Charge < 0:
                        cut4 = cut4 +1;

                    
                        #Corte 5: muones mas energéticos tengan masa invariante diferente de la del Z
                        MassMuon= (muon1.P4()+muon2.P4()).M();
                        if not((MassMuon>80) and (MassMuon<100)):
                            cut5 = cut5 +1;
    
    
    cuts['C0']=numberOfEntries;
    cuts['C1'] = cut1;
    cuts['C2'] = cut2;
    cuts['C3'] = cut3;
    cuts['C4'] = cut4;
    cuts['C5'] = cut5; 
    signal[Folder]=cuts

TClass::TClass:0: RuntimeWarning: no dictionary for class CompBase is available


Donde Co,C1,C2,... Son el número de eventos, sin cortes, con el primer corte, segundo y asi susecivamente.

In [5]:
signal

{'/BackGround-DY_2j-Run0': {'C0': 160448L,
  'C1': 28247,
  'C2': 317,
  'C3': 137,
  'C4': 137,
  'C5': 14},
 '/BackGround-WW-Run0': {'C0': 85553L,
  'C1': 74134,
  'C2': 25633,
  'C3': 11387,
  'C4': 11387,
  'C5': 9838},
 '/BackGround-mumuW': {'C0': 325964L,
  'C1': 238907,
  'C2': 511,
  'C3': 196,
  'C4': 196,
  'C5': 10},
 '/BackGround-tW-Run0': {'C0': 80417L,
  'C1': 76727,
  'C2': 27754,
  'C3': 12751,
  'C4': 12750,
  'C5': 10985},
 '/Signal_2j_mu-nu-Run0': {'C0': 355148L,
  'C1': 341827,
  'C2': 126190,
  'C3': 55688,
  'C4': 55687,
  'C5': 48077}}

In [21]:
EffA={};  #Eficiencia Acumulada
for entry in signal.keys():
    EffCut={};  
    for i in signal[entry].keys():
        EffCut[i] = (0.1*signal[entry][i])/(0.1*signal[entry]['C0']);
    EffA[entry] = EffCut;

In [50]:
Effr={};  #Eficiencia Relativa
a=signal[entry].keys()
a.sort()
for entry in signal.keys():
    EffCut={};
    j=0;
    a1=a[j];
    for i in a[1:]:
        EffCut[i] = (0.1*signal[entry][i])/(0.1*signal[entry][a1]);
        a1=a[j]
        j=j+1;
    Effr[entry] = EffCut;

In [51]:
Effr

{'/BackGround-DY_2j-Run0': {'C1': 0.17605080773833268,
  'C2': 0.0019757179896290385,
  'C3': 0.0048500725740786635,
  'C4': 0.43217665615141954,
  'C5': 0.10218978102189781},
 '/BackGround-WW-Run0': {'C1': 0.8665271819807604,
  'C2': 0.29961544305868876,
  'C3': 0.15360023740793696,
  'C4': 0.4442320446299692,
  'C5': 0.8639676824448933},
 '/BackGround-mumuW': {'C1': 0.7329244947294793,
  'C2': 0.0015676577781595514,
  'C3': 0.0008204029182903808,
  'C4': 0.3835616438356165,
  'C5': 0.0510204081632653},
 '/BackGround-tW-Run0': {'C1': 0.9541141798375965,
  'C2': 0.34512603056567637,
  'C3': 0.16618660966804386,
  'C4': 0.4593932406139655,
  'C5': 0.8615010587404909},
 '/Signal_2j_mu-nu-Run0': {'C1': 0.9624916936037934,
  'C2': 0.3553166567177627,
  'C3': 0.16291281847250216,
  'C4': 0.44129487281084084,
  'C5': 0.8633278264617152}}

SyntaxError: invalid syntax (<ipython-input-46-febaee6fe491>, line 2)

In [45]:
a1

'C1'

In [22]:
EffA

{'/BackGround-DY_2j-Run0': {'C0': 1.0,
  'C1': 0.17605080773833268,
  'C2': 0.0019757179896290385,
  'C3': 0.000853859194256083,
  'C4': 0.000853859194256083,
  'C5': 8.725568408456322e-05},
 '/BackGround-WW-Run0': {'C0': 1.0,
  'C1': 0.8665271819807604,
  'C2': 0.29961544305868876,
  'C3': 0.1330987808726754,
  'C4': 0.1330987808726754,
  'C5': 0.11499304524680606},
 '/BackGround-mumuW': {'C0': 1.0,
  'C1': 0.7329244947294793,
  'C2': 0.0015676577781595514,
  'C3': 0.0006012933943625677,
  'C4': 0.0006012933943625677,
  'C5': 3.067823440625345e-05},
 '/BackGround-tW-Run0': {'C0': 1.0,
  'C1': 0.9541141798375965,
  'C2': 0.34512603056567637,
  'C3': 0.15856100078341645,
  'C4': 0.1585485656018006,
  'C5': 0.13660047004986506},
 '/Signal_2j_mu-nu-Run0': {'C0': 1.0,
  'C1': 0.9624916936037934,
  'C2': 0.3553166567177627,
  'C3': 0.15680223456136597,
  'C4': 0.1567994188338383,
  'C5': 0.1353717323482041}}